##### Author: KAZI ABIR ADNAN | i_know1988@yahoo.com

## README

- This notebook expects an input file named 'input.xlsx' in the root directory
- If you run all cells, this should an 'output_final.csv' file in the root folder

## Problem Description & Solution

### Problem

#### Coding test for Caltex Big Data Engineer Interview. 

### Solution

- Step 1 : Load data from excel file using pandas
- Step 2 : Added a column of hierarchical information of products looping through the dataframe.
- Step 3 : Created product hierarchy columns using lambda function
- Step 4 : Unpivoted the dataframe
- Step 5 : Added periodical information using lamda function

## Import Libraries 

In [1]:
import pandas as pd
from datetime import datetime

## Load data from source

In [2]:
df = pd.read_excel('input.xlsx')

In [3]:
df.head(2)

,Unnamed: 0,Unnamed: 1,Units,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,...,2020-11-01 00:00:00,2020-12-01 00:00:00,Q1-21,Q2-21,Q3-21,Q4-21,Q1-22,Q2-22,Q3-22,Q4-22
0,Product,CRUDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Physical,Physical Premiums,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Preprocessing

#### Renaming first two columns

In [4]:
df = df.rename(columns= {"Unnamed: 0": 'category',"Unnamed: 1": 'product_type'})

#### Adding combined hierarchy column and populate hierarchy information

In [5]:
df['combined_hierarchy'] = None

The hierarchical information maintained here is not thread-safe and dataset dependent. There is a scope to improve this method. I wasn't able to use lambda method as I need to access previous rows to get the hierarchy. Moreover, hierarchy level suddenly changes from level 4 to level 2 and other than using 'Singapore' value in product_type it is hard to capture

In [6]:
def add_combined_hierarchy_value(df):
    """ 
    Updates the dataframe defined above with hierarchical information
    
    Extended description of function. 
  
    Parameters: 
    arg1 (dataframe): takes a dataframe 
  
    Returns: 
    dataframe : updated dataframe with combined_hierarchy seprated by comma in order
    """
    
    for index, row in df.iterrows():
        if row['category'] == 'Product':
            #reset hierarchy
            hierarchy = []
        if pd.isnull(row['Units']):
            # TODO: THERE IS AN IMPROVEMENT SCOPE HERE. CURRENT CODE WILL ONLY WORK FOR THIS DATASET
            if (len(hierarchy) == 4) and hierarchy[-1] != "Singapore":
                hierarchy.pop()
            elif len(hierarchy) > 0 and hierarchy[-1] == "Singapore":
                hierarchy.pop()
                hierarchy.pop()
            hierarchy.append(row['product_type'])
        else:
            combined = ','.join(hierarchy)
            combined = combined + "," + row['product_type']
            df.at[index, 'combined_hierarchy'] = combined
    
    return df

#### Create all hierarchy columns

In [7]:
def get_product_hierarchy(row):
    """ 
    Takes a row of the dataframe and updates the row with product hierarchy information. 
  
    Extended description of function. 
  
    Parameters: 
    arg1 (row): dataframe row
  
    Returns: 
    row: updated dataframe row
  
    """
    value = row['combined_hierarchy']
    if value:
        values = value.split(',')
        row['product_name'] = values[0]
        for i in range(1, len(values)):
            row['product_hier_'+str(i)] = values[i]
    else:
        row['product_name'] = None
        row['product_hier_1'] = None
        row['product_hier_2'] = None
        row['product_hier_3'] = None 
        row['product_hier_4'] = None 

    return row

#### Add product_name, product_hier_1, product_hier_2, product_hier_3, product_hier_4 columns to dataframe from combined_hierarchy

In [8]:
df = add_combined_hierarchy_value(df)
df = df.apply(get_product_hierarchy, axis = 1)

#### Dropping unnecessary columns and rows

In [9]:
df = df.drop(['combined_hierarchy','product_type'], axis=1)
df = df.dropna(subset = ['Units'])

## Data Manipulation

#### Unpivoting table

In [10]:
id_vars_column = ['product_name', 'product_hier_1','product_hier_2','product_hier_3','product_hier_4',
                  "category","Units"]
value_vars_name = list(df.columns[~df.columns.isin(id_vars_column)])
df = pd.melt(df, id_vars = id_vars_column, value_vars = value_vars_name)

#### Adding periodical information

In [11]:
#TODO: IMPROVEMENT SCOPE - replace year_value map and use a method to get 4 digit year from 2 digit year
year_value = {'21': 2021, '22': 2022}
def period_type(row):
    """ 
    Takes a row of the dataframe and updates the row with periodical information. 
  
    Extended description of function. 
  
    Parameters: 
    arg1 (row): dataframe row
  
    Returns: 
    row: updated dataframe row
  
    """
    
    value = str(row['variable'])
    if value.startswith('Q'):
        row['period_type'] = 'quarter'
        row['period'] = int(value[1])
        row['year'] = year_value[value.split('-')[1]]
    else:
        dt = datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
        row['period_type'] = 'month'
        row['period'] = dt.month
        row['year'] = dt.year
    
    return row

#### Add period_type, period, year columns to dataframe from variable column

In [12]:
df = df.apply(period_type, axis = 1)

In [13]:
df.head(2)

,product_name,product_hier_1,product_hier_2,product_hier_3,product_hier_4,category,Units,variable,value,period_type,period,year
0,CRUDE,Physical Premiums,Regional vs Dated Brent,NaN,NaN,Physical,bbl,2019-06-01 00:00:00,NaN,month,6,2019
1,CRUDE,Physical Premiums,US vs Dated Brent,NaN,NaN,Physical,bbl,2019-06-01 00:00:00,NaN,month,6,2019


## Data Saving

#### Processing to match output format

In [14]:
df = df[['category', 'product_name', 'product_hier_1', 'product_hier_2', 'product_hier_3', 'product_hier_4',
        'year','period_type','period', 'Units', 'value']]
df = df.rename(columns = {'Units': 'units'})

#### Write to file

In [15]:
df.to_csv('output_final.csv', index = False)